In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Serving PyTorch image models with prebuilt containers on Vertex AI


<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/pytorch_image_classification_with_prebuilt_serving_containers.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/pytorch_image_classification_with_prebuilt_serving_containers.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/prediction/pytorch_image_classification_with_prebuilt_serving_containers.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

This tutorial demonstrates how to upload and deploy a PyTorch image model using a prebuilt serving container and how to make online and batch predictions.

Vertex AI provides prebuilt containers for serving predictions and explanations from trained model artifacts. Using a pre-built container is generally simpler than creating your own custom container for prediction.

Learn more about [Pre-built containers for prediction](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers).

### Objective

In this tutorial, you learn how to package and deploy a PyTorch image classification model using a prebuilt Vertex AI container with TorchServe for serving online and batch predictions.

This tutorial uses the following Google Cloud ML services and resources:

- `Vertex AI Model Registry`
- `Vertex AI Model` resources
- `Vertex AI Endpoint` resources

The steps performed include:

- Download a pretrained image model from PyTorch
- Create a custom model handler
- Package model artifacts in a model archive file
- Upload model for deployment
- Deploy model for prediction
- Make online predictions
- Make batch predictions

### Model

This tutorial uses a pretrained image model [resnet18](https://pytorch.org/vision/master/models/generated/torchvision.models.resnet18.html) from the PyTorch TorchVision.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [3]:
! pip3 install --upgrade --quiet google-cloud-aiplatform==1.23.0 \
                                 # tensorflow==2.12.0 \
                                 torch==2.0.0 \
                                 torchvision==0.15.1 \
                                 torch-model-archiver==0.7.1 \
                                 packaging==14.3

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, see the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [1]:
PROJECT_ID = "tom-zhang-0"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [3]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [4]:
! gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=MA3975lSLx2PnLIDZRlEJmHwX9Nvt0&access_type=offline&code_challenge=lNqzn_W37ApNgXktJHN_g9TbZujINO4YXqf4MqRzoJI&code_challenge_method=S256


You are now logged in as [szhang1@g.harvard.edu].
Your current project is [tom-zhang-0].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


**3. Colab, uncomment and run:**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [5]:
BUCKET_URI = f"gs://{PROJECT_ID}-test-bucket"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [62]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://tom-zhang-0-test-bucket/...


### Import libraries

In [6]:
import base64
import json
import os
import pathlib
import urllib.request

# import tensorflow as tf
import torch
from google.cloud import aiplatform
from PIL import Image
from torchvision import models

/opt/anaconda3/envs/ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [7]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Download a pre-trained image model

Download the pretrained image model [resnet18](https://pytorch.org/vision/master/models/generated/torchvision.models.resnet18.html) from the PyTorch TorchVision.

In [8]:
# Create a local directory for model artifacts
model_path = "model"

!rm -r $model_path
!mkdir $model_path

model_name = "distilgpt2-custom-handler"
model_file = f"{model_path}/{model_name}.pt"

In [9]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load distilgpt-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

# Save the tokenizer and model to the local directory
tokenizer.save_pretrained(model_path)
model.save_pretrained(model_path)

Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.


In [10]:
model_name = "pytorch_model"
model_file = f"{model_path}/{model_name}.bin"

In [24]:
# # Use scripted mode to save the PyTorch model locally
# model = models.resnet18(pretrained=True)
# script_module = torch.jit.script(model)
# script_module.save(model_file)

## Create a custom model handler

A custom model handler is a Python script that you package with the model when you use the model archiver. The script typically defines how to pre-process input data, invoke the model and post-process the output.

TorchServe has [default handlers](https://pytorch.org/serve/default_handlers.html) for `image_classifier`, `image_segmenter`, `object_detector` and `text_classifier`. In this tutorial, you create a custom handler extending the default [`image_classifier`](https://github.com/pytorch/serve/blob/master/ts/torch_handler/image_classifier.py) handler.


In [15]:
hander_file = f"{model_path}/custom_handler.py"

In [ ]:
%%writefile {hander_file}

import torch
import torch.nn.functional as F
from ts.torch_handler.image_classifier import ImageClassifier
from ts.utils.util import map_class_to_label


class CustomImageClassifier(ImageClassifier):

    # Only return the top 3 predictions
    topk = 3

    def postprocess(self, data):
        ps = F.softmax(data, dim=1)
        probs, classes = torch.topk(ps, self.topk, dim=1)
        probs = probs.tolist()
        classes = classes.tolist()
        return map_class_to_label(probs, self.mapping, classes)

## Download an index_to_name.json file

PyTorch `image_classifier`, `text_classifier` and `object_detector` can all automatically map from numeric classes (0,1,2...) to friendly strings. To do this, simply include `index_to_name.json` that contains a mapping of class number to friendly name in your model archive file.

In [ ]:
index_to_name_file = f"{model_path}/index_to_name.json"

urllib.request.urlretrieve(
    "https://github.com/pytorch/serve/raw/master/examples/image_classifier/index_to_name.json",
    index_to_name_file,
)

## Package the model artifacts in a model archive file

You package all the model artifacts in a model archive file using the [`Torch model archiver`](https://github.com/pytorch/serve/tree/master/model-archiver).

Note that the prebuilt PyTorch serving containers require the model archive file named as `model.mar` so you need to set the model-name as `model` in the `torch-model-archiver` command.

In [11]:
# Add torch-model-archiver to the PATH
os.environ["PATH"] = f'{os.environ.get("PATH")}:~/.local/bin'

In [12]:
!torch-model-archiver -f \
  --model-name model \
  --version 1.0  \
  --serialized-file $model_file \
  --handler custom_handler.py \
  --extra-files model/config.json,model/vocab.json,model/generation_config.json \
  --export-path $model_path

## Copy the model artifacts to Cloud Storage

Next, use `gsutil` to copy the model artifacts to your Cloud Storage bucket.

In [13]:
MODEL_URI = f"{BUCKET_URI}/{model_name}"

!gsutil rm -r $MODEL_URI
!gsutil cp -r $model_path $MODEL_URI
!gsutil ls -al $MODEL_URI

Removing gs://tom-zhang-0-test-bucket/pytorch_model/added_tokens.json#1698085166525435...
Removing gs://tom-zhang-0-test-bucket/pytorch_model/config.json#1698085167280563...
Removing gs://tom-zhang-0-test-bucket/pytorch_model/generation_config.json#1698085167506621...
Removing gs://tom-zhang-0-test-bucket/pytorch_model/merges.txt#1698085168009398...
/ [4 objects]                                                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rm ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Removing gs://tom-zhang-0-test-bucket/pytorch_model/model.mar#1698085166264136...
Removing gs://tom-zhang-0-test-bucket/pytorch_model/pytorch_model.bin#1698085257684708...
Removing gs://tom-zhang-0-test-bucket/pytorch_model/special_tokens_map.json#1698085166976833...
Removing gs://tom-zhang-0-test-bucket/pytorch_m

## Upload model for deployment

Next, you upload the [model](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.models) to `Vertex AI Model Registry`, which will create a `Vertex AI Model` resource for your model. This tutorial uses the PyTorch v1.11 container, but for your own use case, you can choose from the list of [PyTorch prebuilt containers](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers#pytorch).

In [14]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'secrets/hisolver-data-collection-secrets.json'

In [15]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

note: need aiplatform upload permission

In [84]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:hisolver-data-collection@tom-zhang-0.iam.gserviceaccount.com \
    --role=roles/aiplatform.admin

Updated IAM policy for project [tom-zhang-0].
bindings:
- members:
  - serviceAccount:hisolver-data-collection@tom-zhang-0.iam.gserviceaccount.com
  - user:szhang1@g.harvard.edu
  role: roles/aiplatform.admin
- members:
  - serviceAccount:service-113408214330@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-113408214330@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:service-113408214330@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-113408214330@containerregistry.iam.gserviceaccount.com
  role: roles/containerregistry.ServiceAgent
- members:
  - serviceAccount:service-113408214330@dataflow-service-producer-prod.iam.gserviceaccount.com
  role: roles/dataflow.serviceAgent
- members:
  - serviceAccount:113408214330-compute@developer.gserviceaccount.com
  - serviceAccount:11340821433

In [81]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=user:szhang1@g.harvard.edu \
    --role=roles/aiplatform.admin

Updated IAM policy for project [tom-zhang-0].
bindings:
- members:
  - user:szhang1@g.harvard.edu
  role: roles/aiplatform.admin
- members:
  - serviceAccount:service-113408214330@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-113408214330@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:service-113408214330@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-113408214330@containerregistry.iam.gserviceaccount.com
  role: roles/containerregistry.ServiceAgent
- members:
  - serviceAccount:service-113408214330@dataflow-service-producer-prod.iam.gserviceaccount.com
  role: roles/dataflow.serviceAgent
- members:
  - serviceAccount:113408214330-compute@developer.gserviceaccount.com
  - serviceAccount:113408214330@cloudservices.gserviceaccount.com
  role: roles/editor
- members:
  - serviceA

In [88]:
!gcloud iam roles describe --project=$PROJECT_ID aiplatform.admin

ERROR: (gcloud.iam.roles.describe) NOT_FOUND: The role named projects/tom-zhang-0/roles/aiplatform.admin was not found.


In [16]:
DEPLOY_IMAGE_URI = "us-docker.pkg.dev/vertex-ai/prediction/pytorch-cpu.1-11:latest"

deployed_model = aiplatform.Model.upload(
    display_name=model_name,
    serving_container_image_uri=DEPLOY_IMAGE_URI,
    artifact_uri=MODEL_URI,
)

Creating Model
Create Model backing LRO: projects/113408214330/locations/us-central1/models/1963910286138146816/operations/6110633109424701440
Model created. Resource name: projects/113408214330/locations/us-central1/models/1963910286138146816@1
To use this Model in another session:
model = aiplatform.Model('projects/113408214330/locations/us-central1/models/1963910286138146816@1')


## Deploy model for prediction

Next, deploy your model for online prediction. You set the variable `DEPLOY_COMPUTE` to configure the machine type for the [compute resources](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute) you will use for prediction.

In [ ]:
# this may take a while (10 min?)

In [18]:
DEPLOY_COMPUTE = "n1-standard-4"

endpoint = deployed_model.deploy(
    deployed_model_display_name=model_name,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=None,
    accelerator_count=0,
)

Creating Endpoint
Create Endpoint backing LRO: projects/113408214330/locations/us-central1/endpoints/3418463014616039424/operations/8549895257599246336
Endpoint created. Resource name: projects/113408214330/locations/us-central1/endpoints/3418463014616039424
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/113408214330/locations/us-central1/endpoints/3418463014616039424')
Deploying model to Endpoint : projects/113408214330/locations/us-central1/endpoints/3418463014616039424
Deploy Endpoint model backing LRO: projects/113408214330/locations/us-central1/endpoints/3418463014616039424/operations/5499832409962577920
Endpoint model deployed. Resource name: projects/113408214330/locations/us-central1/endpoints/3418463014616039424


In [19]:
endpoint

resource name: projects/113408214330/locations/us-central1/endpoints/3418463014616039424

In [20]:
endpoint.predict(instances=[{'data': "test data"}])

ServiceUnavailable: 503 Took too long to respond when processing endpoint_id: 3418463014616039424, deployed_model_id: 2319980529723637760

## Make online predictions

### Download an image dataset
In this example, you use the TensorFlow flowers dataset for the input for both online and batch predictions.

In [ ]:
data_dir = tf.keras.utils.get_file(
    "flower_photos",
    origin="https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
    untar=True,
)

data_dir = pathlib.Path(data_dir)
images_files = list(data_dir.glob("daisy/*"))

### Get online predictions

You send a `predict` request with encoded input image data to the `endpoint` and get prediction.

In [ ]:
with open(images_files[0], "rb") as f:
    data = {"data": base64.b64encode(f.read()).decode("utf-8")}

response = endpoint.predict(instances=[data])

In [ ]:
prediction = response.predictions[0]
prediction = dict(sorted(prediction.items(), key=lambda item: item[1], reverse=True))

print(prediction)
image = Image.open(images_files[0])
image

## Make batch predictions

### Create the batch input file

You create a batch input file in JSONL format and store the input file in your Cloud Storage bucket.

Learn more about [Input data requirements](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions#input_data_requirements).

In [ ]:
TEST_IMAGE_SIZE = 2
test_image_list = []
for i in range(TEST_IMAGE_SIZE):
    test_image_list.append(str(images_files[i]))

gcs_input_uri = f"{BUCKET_URI}/test_images.json"

with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for test_image in test_image_list:
        with open(test_image, "rb") as image_f:
            data = {"data": base64.b64encode(image_f.read()).decode("utf-8")}
            f.write(json.dumps(data) + "\n")

### Submit a batch prediction job

In [ ]:
JOB_DISPLAY_NAME = f"{model_name}_batch_predict_job_unique"

batch_predict_job = deployed_model.batch_predict(
    job_display_name=JOB_DISPLAY_NAME,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
)

### Get batch predictions

After the batch job completes, the results are written to the Cloud Storage output bucket you specified in the batch request. You call the method `iter_outputs()` to get a list of each Cloud Storage file generated with the results.

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_files = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction.results"):
        prediction_files.append(blob.name)

In [ ]:
prediction_file = prediction_files[0]

results = []
gfile_name = f"{BUCKET_URI}/{prediction_file}"
with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
    for line in gfile.readlines():
        results.append(json.loads(line))

# Take one result as an example and print out the prediction.
prediction = results[0]["prediction"]
prediction = dict(sorted(prediction.items(), key=lambda item: item[1], reverse=True))
print(prediction)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

deployed_model.delete()
batch_predict_job.delete()

delete_bucket = False
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI